# Download Dataset

In [1]:
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'

In [12]:
import os

datasets_path = os.path.join('..', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')

In [2]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9676 sha256=e2ae3e18c748d1523ea2d5e3bd6b204adc0be82591308bff67a490ae41f0ff91
  Stored in directory: /root/.cache/pip/wheels/04/5f/3e/46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [14]:
import wget

complete_zip = wget.download(complete_dataset_url)
small_zip = wget.download(small_dataset_url)

In [5]:
import zipfile

with zipfile.ZipFile('ml-latest-small.zip', "r") as z:
    z.extractall('')

In [6]:
with zipfile.ZipFile('ml-latest.zip', "r") as z:
    z.extractall('')

In [8]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 18.4 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=36337be55257d4049082ab6c3762f5017c419cacd1ca5721f90e767b54132efd
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [16]:
from pyspark import SparkContext
sc = SparkContext("local", "First App")

In [19]:
small_ratings_raw_data = sc.textFile("ml-latest-small/ratings.csv")

small_ratings_raw_data

ml-latest-small/ratings.csv MapPartitionsRDD[6] at textFile at NativeMethodAccessorImpl.java:0

In [20]:
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]
small_ratings_raw_data_header

'userId,movieId,rating,timestamp'

In [21]:
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [22]:
small_ratings_data.take(3)

[('1', '1', '4.0'), ('1', '3', '4.0'), ('1', '6', '4.0')]

In [24]:
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=99)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [36]:
from pyspark.mllib.recommendation import ALS
import math

seed = 99
iterations = [5, 10, 20]
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
  for iteration in iterations:
    model = ALS.train(training_RDD, rank, seed=seed, iteration,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print ('For rank', rank, 'the RMSE is', error)
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank', best_rank)

SyntaxError: ignored

In [ ]:
predictions.take(3)

In [ ]:
rates_and_preds.take(3)

In [ ]:
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print 'For testing data the RMSE is %s' % (error)